In [1]:
from __future__ import print_function, division
import pandas as pd
import requests
import pickle
from pandas.core import datetools

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

%matplotlib inline

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  """


In [2]:
pd.set_option('display.max_columns', 50)

# Data Scraping

In [ ]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
chromedriver_path = "/Users/Tong/chromedriver"
driver = webdriver.Chrome(chromedriver_path)
driver.get('https://www.whoscored.com/')
time.sleep(5); 

In [ ]:
def Laliga_click():
    Laliga = driver.find_element_by_xpath('//a[@title="Spain"]')
    Laliga.click()
    time.sleep(5);

In [ ]:
Laliga_click() #choose the league Laliga

Get match results for all the matches in 7 seasons.

Get team statistical data for 7 seasons(2010-2011, 2011-2012, 2012-2013, 2013-2014, 2014-2015, 2015-2016, 2016-2017). 

Team statistical data consists of three aspects: summary stats, defensive stats and offensive stats.

Team statistics are different for a team when it's playing at home or away. 

# Clean Data

# Data Exploration

In [4]:
Goals_vs_Statistics = pickle.load(open('data/Goals_vs_Statistics.p', 'rb'))
Goals_vs_Statistics.head()

,season,year,month,home_team,away_team,home_team_goals,away_team_goals,h_shots_pg,h_discipline_yellow,h_discipline_red,h_possession,h_pass_success,h_aerials_won,h_shots_conceded_pg,h_tackles_pg,h_interceptions_pg,h_fouls_pg,h_offsides_pg,h_shots_ot_pg,h_dribbles_pg,h_fouled_pg,a_shots_pg,a_discipline_yellow,a_discipline_red,a_possession,a_pass_success,a_aerials_won,a_shots_conceded_pg,a_tackles_pg,a_interceptions_pg,a_fouls_pg,a_offsides_pg,a_shots_ot_pg,a_dribbles_pg,a_fouled_pg
0,2010 - 2011,1,8,Hercules,Athletic Club,0,1,12.7,37,5,50.9,76.7,15.6,13.8,18.9,29.3,13.3,3.7,4.8,5.3,14.1,12.0,52,2,47.6,73.8,15.7,13.9,21.7,26.0,16.0,3.4,4.8,5.1,13.2
1,2010 - 2011,1,8,Malaga,Valencia,1,3,13.8,47,1,51.1,80.1,11.7,13.8,19.5,25.2,14.3,2.1,4.7,7.2,15.6,11.1,62,7,54.1,80.2,11.1,15.1,17.3,24.4,15.5,2.3,4.4,7.6,12.5
2,2010 - 2011,1,8,Levante,Sevilla,1,4,11.8,62,1,45.9,71.3,13.1,12.6,19.1,24.8,15.4,2.5,3.9,6.4,13.5,12.1,55,5,52.0,79.5,12.8,13.6,19.7,20.5,14.8,2.4,4.7,6.9,14.7
3,2010 - 2011,1,8,Real Sociedad,Villarreal,1,0,12.5,38,0,47.8,71.3,16.5,14.9,20.4,24.2,14.4,2.8,4.3,4.8,15.2,12.5,45,5,53.6,83.9,7.8,13.3,22.8,23.1,12.4,1.9,4.4,8.5,14.9
4,2010 - 2011,1,8,Mallorca,Real Madrid,0,0,14.1,39,0,47.3,75.0,15.7,10.9,19.6,24.3,14.2,3.1,5.6,7.6,10.9,17.8,53,3,53.9,81.3,9.9,10.9,24.5,21.6,14.0,4.1,7.2,11.4,13.9


In [5]:
Goals_vs_Statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2650 entries, 0 to 2649
Data columns (total 35 columns):
season                 2650 non-null object
year                   2650 non-null object
month                  2650 non-null object
home_team              2650 non-null object
away_team              2650 non-null object
home_team_goals        2650 non-null int64
away_team_goals        2650 non-null int64
h_shots_pg             2650 non-null float64
h_discipline_yellow    2650 non-null int64
h_discipline_red       2650 non-null int64
h_possession           2650 non-null float64
h_pass_success         2650 non-null float64
h_aerials_won          2650 non-null float64
h_shots_conceded_pg    2650 non-null float64
h_tackles_pg           2650 non-null float64
h_interceptions_pg     2650 non-null float64
h_fouls_pg             2650 non-null float64
h_offsides_pg          2650 non-null float64
h_shots_ot_pg          2650 non-null float64
h_dribbles_pg          2650 non-null float64
h_foul

In [6]:
#split data into test_set and train_ste
Test_set = Goals_vs_Statistics[Goals_vs_Statistics['season'] == '2016 - 2017']
Test_set.shape

(380, 35)

In [7]:
Train_set = Goals_vs_Statistics[Goals_vs_Statistics['season'] != '2016 - 2017']
Train_set.shape

(2270, 35)

In [8]:
#Let's see the correlations between the number of home team goals with diffrent variables
Train_set.corr().sort_values('home_team_goals')

,home_team_goals,away_team_goals,h_shots_pg,h_discipline_yellow,h_discipline_red,h_possession,h_pass_success,h_aerials_won,h_shots_conceded_pg,h_tackles_pg,h_interceptions_pg,h_fouls_pg,h_offsides_pg,h_shots_ot_pg,h_dribbles_pg,h_fouled_pg,a_shots_pg,a_discipline_yellow,a_discipline_red,a_possession,a_pass_success,a_aerials_won,a_shots_conceded_pg,a_tackles_pg,a_interceptions_pg,a_fouls_pg,a_offsides_pg,a_shots_ot_pg,a_dribbles_pg,a_fouled_pg
h_shots_conceded_pg,-0.241336,0.133993,-0.354425,0.102724,0.147389,-0.532104,-0.458180,0.102570,1.000000,-0.083118,0.254845,0.309906,-0.185573,-0.419850,-0.520510,0.060997,0.067748,-0.019861,0.023093,0.034124,0.023172,-0.098358,0.025704,0.092313,0.066372,0.003698,0.074199,0.065855,-0.053818,0.063866
h_fouls_pg,-0.231836,0.048274,-0.316481,0.266169,0.221695,-0.534912,-0.638594,0.267400,0.309906,0.175964,0.314299,1.000000,-0.175425,-0.399241,-0.504666,-0.015960,0.033751,-0.101060,-0.029625,0.023029,0.007468,-0.149057,0.048337,0.112315,0.158276,0.009677,0.086226,0.060415,-0.072285,0.102812
h_aerials_won,-0.218045,0.039875,-0.492698,0.339703,0.349140,-0.448585,-0.556459,1.000000,0.102570,-0.096339,-0.284137,0.267400,-0.355028,-0.491343,-0.336588,-0.061994,0.007129,0.183313,0.034912,0.026712,0.055437,0.310264,-0.123968,-0.184335,-0.445403,-0.081493,-0.154561,-0.052085,0.179162,-0.127427
a_shots_ot_pg,-0.145800,0.347313,0.005599,-0.004753,0.011115,-0.039738,-0.041157,-0.052085,0.065855,0.052308,0.092487,0.060415,0.018004,-0.007781,-0.083619,0.014430,0.858985,-0.219699,-0.263468,0.745588,0.743947,-0.413139,-0.536646,-0.007064,-0.212661,-0.418540,0.514928,1.000000,0.589585,0.402854
a_pass_success,-0.135742,0.291277,-0.022295,-0.065617,-0.027382,-0.036204,-0.028343,0.055437,0.023172,-0.036895,-0.050487,0.007468,-0.038429,-0.041028,-0.011479,-0.044077,0.695873,-0.109074,-0.153882,0.917148,1.000000,-0.484327,-0.518897,-0.167566,-0.336927,-0.545839,0.325746,0.743947,0.747244,0.423011
a_possession,-0.135535,0.290048,-0.016590,-0.044262,-0.017497,-0.044059,-0.035270,0.026712,0.034124,-0.002669,0.001601,0.023029,-0.023160,-0.035310,-0.034537,-0.025856,0.696377,-0.090395,-0.164724,1.000000,0.917148,-0.435119,-0.558136,-0.099532,-0.285459,-0.477930,0.431104,0.745588,0.691257,0.456143
a_dribbles_pg,-0.129584,0.259448,-0.122871,0.012155,-0.008044,-0.031962,-0.017749,0.179162,-0.053818,-0.138710,-0.215013,-0.072285,-0.109393,-0.100861,0.084687,-0.063054,0.574170,-0.051507,-0.115781,0.691257,0.747244,-0.274628,-0.522303,-0.179191,-0.441587,-0.466309,0.210185,0.589585,1.000000,0.275944
h_discipline_yellow,-0.125676,0.045518,-0.426114,1.000000,0.666586,-0.306776,-0.339174,0.339703,0.102724,0.049828,-0.307140,0.266169,-0.134363,-0.262564,-0.167527,0.114998,-0.081927,-0.292319,-0.159591,-0.044262,-0.065617,0.239478,-0.027697,0.072158,-0.347116,0.048135,-0.065872,-0.004753,0.012155,0.013871
h_interceptions_pg,-0.118255,0.021017,0.024985,-0.307140,-0.229086,-0.209623,-0.286903,-0.284137,0.254845,0.400404,1.000000,0.314299,0.069702,-0.134004,-0.448858,0.163221,0.010396,-0.229052,-0.052823,0.001601,-0.050487,-0.495340,0.133575,0.314407,0.733036,0.105092,0.203290,0.092487,-0.215013,0.220290
a_shots_pg,-0.115044,0.280502,0.017439,-0.081927,-0.010127,-0.018274,-0.016381,0.007129,0.067748,0.038908,0.010396,0.033751,-0.002655,-0.016471,-0.067626,-0.043770,1.000000,0.031980,-0.120657,0.696377,0.695873,-0.280661,-0.453094,0.005338,-0.270223,-0.405259,0.412037,0.858985,0.574170,0.340200


In [9]:
print(Train_set.corr().sort_values('home_team_goals').index)

#the correlation between home_team_goals and the following variables are over 0.1
#['h_shots_conceded_pg', 'h_fouls_pg', 'h_aerials_won', 'a_shots_ot_pg', 'a_pass_success', 'a_possession', 'a_dribbles_pg', 'h_discipline_yellow', 'h_interceptions_pg', 'a_shots_pg', 'h_discipline_red', 'a_shots_conceded_pg', 'h_offsides_pg', 'h_dribbles_pg', 'h_possession', 'h_shots_pg', 'h_pass_success', 'h_shots_ot_pg']

Index(['h_shots_conceded_pg', 'h_fouls_pg', 'h_aerials_won', 'a_shots_ot_pg',
       'a_pass_success', 'a_possession', 'a_dribbles_pg',
       'h_discipline_yellow', 'h_interceptions_pg', 'a_shots_pg',
       'h_discipline_red', 'away_team_goals', 'a_offsides_pg', 'a_fouled_pg',
       'h_tackles_pg', 'a_tackles_pg', 'h_fouled_pg', 'a_aerials_won',
       'a_discipline_yellow', 'a_discipline_red', 'a_interceptions_pg',
       'a_fouls_pg', 'a_shots_conceded_pg', 'h_offsides_pg', 'h_dribbles_pg',
       'h_possession', 'h_shots_pg', 'h_pass_success', 'h_shots_ot_pg',
       'home_team_goals'],
      dtype='object')


In [10]:
Train_set.corr().sort_values('h_shots_pg')

#h_shots_pg is highly correlated with other variables, such as: h_possession, h_pass_success, h_shots_pg

,home_team_goals,away_team_goals,h_shots_pg,h_discipline_yellow,h_discipline_red,h_possession,h_pass_success,h_aerials_won,h_shots_conceded_pg,h_tackles_pg,h_interceptions_pg,h_fouls_pg,h_offsides_pg,h_shots_ot_pg,h_dribbles_pg,h_fouled_pg,a_shots_pg,a_discipline_yellow,a_discipline_red,a_possession,a_pass_success,a_aerials_won,a_shots_conceded_pg,a_tackles_pg,a_interceptions_pg,a_fouls_pg,a_offsides_pg,a_shots_ot_pg,a_dribbles_pg,a_fouled_pg
h_aerials_won,-0.218045,0.039875,-0.492698,0.339703,0.349140,-0.448585,-0.556459,1.000000,0.102570,-0.096339,-0.284137,0.267400,-0.355028,-0.491343,-0.336588,-0.061994,0.007129,0.183313,0.034912,0.026712,0.055437,0.310264,-0.123968,-0.184335,-0.445403,-0.081493,-0.154561,-0.052085,0.179162,-0.127427
h_discipline_yellow,-0.125676,0.045518,-0.426114,1.000000,0.666586,-0.306776,-0.339174,0.339703,0.102724,0.049828,-0.307140,0.266169,-0.134363,-0.262564,-0.167527,0.114998,-0.081927,-0.292319,-0.159591,-0.044262,-0.065617,0.239478,-0.027697,0.072158,-0.347116,0.048135,-0.065872,-0.004753,0.012155,0.013871
h_shots_conceded_pg,-0.241336,0.133993,-0.354425,0.102724,0.147389,-0.532104,-0.458180,0.102570,1.000000,-0.083118,0.254845,0.309906,-0.185573,-0.419850,-0.520510,0.060997,0.067748,-0.019861,0.023093,0.034124,0.023172,-0.098358,0.025704,0.092313,0.066372,0.003698,0.074199,0.065855,-0.053818,0.063866
h_fouls_pg,-0.231836,0.048274,-0.316481,0.266169,0.221695,-0.534912,-0.638594,0.267400,0.309906,0.175964,0.314299,1.000000,-0.175425,-0.399241,-0.504666,-0.015960,0.033751,-0.101060,-0.029625,0.023029,0.007468,-0.149057,0.048337,0.112315,0.158276,0.009677,0.086226,0.060415,-0.072285,0.102812
h_discipline_red,-0.110556,0.067618,-0.259430,0.666586,1.000000,-0.249589,-0.283343,0.349140,0.147389,0.021732,-0.229086,0.221695,-0.151351,-0.177608,-0.176586,0.098652,-0.010127,-0.119836,-0.053511,-0.017497,-0.027382,0.161813,0.010979,0.073012,-0.276198,0.013772,-0.044513,0.011115,-0.008044,0.002049
a_aerials_won,0.044107,-0.150157,-0.184712,0.239478,0.161813,0.006963,0.032130,0.310264,-0.098358,-0.178946,-0.495340,-0.149057,-0.135677,-0.096445,0.187727,-0.102736,-0.280661,0.320162,0.153868,-0.435119,-0.484327,1.000000,0.110274,-0.037374,-0.297096,0.175462,-0.363952,-0.413139,-0.274628,-0.409651
a_dribbles_pg,-0.129584,0.259448,-0.122871,0.012155,-0.008044,-0.031962,-0.017749,0.179162,-0.053818,-0.138710,-0.215013,-0.072285,-0.109393,-0.100861,0.084687,-0.063054,0.574170,-0.051507,-0.115781,0.691257,0.747244,-0.274628,-0.522303,-0.179191,-0.441587,-0.466309,0.210185,0.589585,1.000000,0.275944
away_team_goals,-0.106907,1.000000,-0.065863,0.045518,0.067618,-0.083965,-0.086480,0.039875,0.133993,-0.013446,0.021017,0.048274,-0.067556,-0.099061,-0.086990,-0.000870,0.280502,-0.075983,-0.108416,0.290048,0.291277,-0.150157,-0.219868,-0.019314,-0.118711,-0.182347,0.190953,0.347313,0.259448,0.141705
a_pass_success,-0.135742,0.291277,-0.022295,-0.065617,-0.027382,-0.036204,-0.028343,0.055437,0.023172,-0.036895,-0.050487,0.007468,-0.038429,-0.041028,-0.011479,-0.044077,0.695873,-0.109074,-0.153882,0.917148,1.000000,-0.484327,-0.518897,-0.167566,-0.336927,-0.545839,0.325746,0.743947,0.747244,0.423011
a_possession,-0.135535,0.290048,-0.016590,-0.044262,-0.017497,-0.044059,-0.035270,0.026712,0.034124,-0.002669,0.001601,0.023029,-0.023160,-0.035310,-0.034537,-0.025856,0.696377,-0.090395,-0.164724,1.000000,0.917148,-0.435119,-0.558136,-0.099532,-0.285459,-0.477930,0.431104,0.745588,0.691257,0.456143


In [11]:
#We first have a look at the linear regression results for home_team_goals with all the variables that have correlation over 0.1.

lm1 = smf.ols('home_team_goals ~ h_shots_conceded_pg + h_fouls_pg + h_aerials_won + a_shots_ot_pg + a_pass_success + a_possession + a_dribbles_pg + h_discipline_yellow + h_interceptions_pg + a_shots_pg + h_discipline_red + a_shots_conceded_pg + h_offsides_pg + h_dribbles_pg + h_possession + h_shots_pg + h_pass_success + h_shots_ot_pg', data=Train_set)
fit1 = lm1.fit()
fit1.summary()

#The R-squared is 0.238. R-squared adjusted is 0.231. 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        home_team_goals   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     38.95
Date:                Wed, 19 Jul 2017   Prob (F-statistic):          1.95e-118
Time:                        12:59:18   Log-Likelihood:                -3699.9
No. Observations:                2270   AIC:                             7438.
Df Residuals:                    2251   BIC:                             7547.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -2.2398      1.395     -1.606      0.108      -4.975       0.496
h_shots_conceded_pg    -0.0318      0.018     -1.746      0.081      -0.067       0.004
h_fouls_pg          -6.982e-05      0.019     -0.004      0.997      -0.037       0.037
h_aerials_won           0.0124      0.011      1.104      0.270      -0.010       0.034
a_shots_ot_pg          -0.1098      0.059     -1.871      0.061      -0.225       0.005
a_pass_success         -0.0134      0.015     -0.890      0.373      -0.043       0.016
a_possession            0.0090      0.014      0.668      0.504      -0.017       0.036
a_dribbles_pg          -0.0074      0.020     -0.369      0.712      -0.046       0.032
h_discipline_yellow    -0.0005      0.001     -0.358      0.721      -0.003       0.002
h_interceptions_pg     -0.0067      0.007     -0.952      0.341      -0.021       0.007
a_shots_pg              0.0247      0.024      1.011      0.312      -0.023       0.073
h_discipline_red       -0.0182      0.013     -1.389      0.165      -0.044       0.008
a_shots_conceded_pg     0.0772      0.016      4.941      0.000       0.047       0.108
h_offsides_pg           0.0507      0.052      0.980      0.327      -0.051       0.152
h_dribbles_pg           0.0045      0.019      0.235      0.815      -0.033       0.042
h_possession           -0.0106      0.013     -0.823      0.411      -0.036       0.015
h_shots_pg             -0.0717      0.028     -2.585      0.010      -0.126      -0.017
h_pass_success          0.0397      0.016      2.453      0.014       0.008       0.071
h_shots_ot_pg           0.4422      0.049      8.963      0.000       0.345       0.539
==============================================================================
Omnibus:                      171.240   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              222.767
Skew:                           0.661   Prob(JB):                     4.23e-49
Kurtosis:                       3.779   Cond. No.                     8.06e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
# Remove the variables have P value over 0.05. 
lm2 = smf.ols('home_team_goals ~ a_shots_conceded_pg + h_shots_pg + h_pass_success + h_shots_ot_pg', data=Train_set)
fit2 = lm2.fit()
fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        home_team_goals   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     167.6
Date:                Wed, 19 Jul 2017   Prob (F-statistic):          7.44e-126
Time:                        12:59:19   Log-Likelihood:                -3713.4
No. Observations:                2270   AIC:                             7437.
Df Residuals:                    2265   BIC:                             7465.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -4.2215      0.543     -7.775      0.000      -5.286      -3.157
a_shots_conceded_pg     0.0966      0.012      8.070      0.000       0.073       0.120
h_shots_pg             -0.0817      0.024     -3.442      0.001      -0.128      -0.035
h_pass_success          0.0431      0.008      5.474      0.000       0.028       0.059
h_shots_ot_pg           0.4509      0.044     10.254      0.000       0.365       0.537
==============================================================================
Omnibus:                      182.825   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              242.447
Skew:                           0.685   Prob(JB):                     2.26e-53
Kurtosis:                       3.829   Cond. No.                     1.66e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# Since h_shots_pg is highly correlated (with the correlation of 0.88) with h_shots_ot_pg, try to remove h_shots_pg. 
lm3 = smf.ols('home_team_goals ~ a_shots_conceded_pg + h_pass_success + h_shots_ot_pg', data=Train_set)
fit3 = lm3.fit()
fit3.summary()
# The removal of h_shots_pg dosen't imporve adj R-squared.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        home_team_goals   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     218.5
Date:                Wed, 19 Jul 2017   Prob (F-statistic):          1.67e-124
Time:                        12:59:20   Log-Likelihood:                -3719.3
No. Observations:                2270   AIC:                             7447.
Df Residuals:                    2266   BIC:                             7469.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -4.4411      0.540     -8.217      0.000      -5.501      -3.381
a_shots_conceded_pg     0.0938      0.012      7.837      0.000       0.070       0.117
h_pass_success          0.0398      0.008      5.084      0.000       0.024       0.055
h_shots_ot_pg           0.3342      0.028     11.912      0.000       0.279       0.389
==============================================================================
Omnibus:                      183.462   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              243.357
Skew:                           0.687   Prob(JB):                     1.43e-53
Kurtosis:                       3.829   Cond. No.                     1.62e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""